In [ ]:
# 라이브러리 임포트
import torch
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Auto-Regressive Pipline
def predict_autoregressive_mc_dropout(model, initial_seq, dest_lat, dest_lon, scaler, max_steps, distance_threshold=0.1, num_mc_samples=10):
    """
    MC Dropout 기반 자가회귀 예측 함수
    model: 학습된 모델 (dropout 포함)
    num_mc_samples: 매 스텝마다 dropout을 적용하여 몇 번 예측할지
    """
    model.train()  # Dropout 활성화 
    input_seq = initial_seq.clone()
    all_preds = []

    for step in range(max_steps):
        input_tensor = input_seq.to(device)

        preds = []
        with torch.no_grad():  # 불필요한 gradient 계산 방지
            for _ in range(num_mc_samples):
                pred = model(input_tensor).squeeze(0).cpu().numpy()
                preds.append(pred)
        
        # MC Dropout 결과 평균
        pred = np.mean(preds, axis=0)
        all_preds.append(pred)

        # 역정규화
        pred_denorm = scaler.inverse_transform(pred.reshape(1, -1))[0][:4]
        pred_lat, pred_lon = pred_denorm[:2]

        # 예측 결과 Logging ----------------------------------------------------------------------
        if(step % 50 == 0):
            print(f"[Step {step+1}] Predicted: ({pred_lat:.5f}, {pred_lon:.5f}) | "
                  f"Target: ({dest_lat:.5f}, {dest_lon:.5f}) | "
                  f"ΔLat: {abs(pred_lat - dest_lat):.5f}, ΔLon: {abs(pred_lon - dest_lon):.5f}")
        # ---------------------------------------------------------------------------------------
        # 목적지 도착 유무 판단 매커니즘 ----------------------------------------------------------
        if abs(pred_lat - dest_lat) < distance_threshold and abs(pred_lon - dest_lon) < distance_threshold:
            print(f"🚢 목적지 도달 - Step: {step + 1}, {int(step/120)} 시간 {int((step%120)/2)} 분 소요")
            break
        # ---------------------------------------------------------------------------------------
        
        # 입력 시퀀스 업데이트
        delta_lat = dest_lat - pred_lat
        delta_lon = dest_lon - pred_lon
        distance = np.sqrt(delta_lat ** 2 + delta_lon ** 2)

        pred_norm = pred.reshape(1, -1)[0]
        pred_norm = np.concatenate([pred_norm, [distance]])
        next_input = pred_norm

        input_seq_np = input_seq.squeeze(0).numpy()
        input_seq_np = np.vstack([input_seq_np[1:], next_input])
        input_seq = torch.tensor([input_seq_np], dtype=torch.float32)

    return np.array(all_preds)